In [1]:
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

mkdir: cannot create directory ‘build’: File exists
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- Found PythonInterp: /home/juampi/HOME/.virtualenvs/tp3_metodos_bis/bin/python (found version "3.6.9") 
-- Found PythonLibs: /usr/lib/x86_64-linux-gnu/libpython3.6m.so
-- pybind11 v2.2.4
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- LTO enabled
CMAKE_INSTALL_PREFIX=/home/juampi/facultad/metodos/tp3_alu
-- Configuri

In [72]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, mean_squared_log_error 
from sklearn.linear_model import LinearRegression
import itertools
from tqdm import tqdm

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [3]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python --version
import metnum


/home/juampi/facultad/metodos/tp3_alu/notebooks
Python 3.6.9


In [4]:
df = pd.read_csv('../data/train.csv')

## Funciones

In [5]:
def crear_matriz_A(array, df):
    
    A = np.zeros(shape=(len(df),array.shape[0]))
    
    for index, elem in enumerate(array):
        col = df[elem].values.reshape(-1, 1)
    
        for i in range(len(df)):
             A[i,index] =float(col[i][0])
    return A

def limpiar(df, cat):
    res_df = df.copy()
    res_df.dropna(subset= cat, inplace=True)
    res_df.reset_index(drop=True, inplace=True)
    
    return res_df


def cross_val_log(df, K, fitting_cat):
    errores = []
    
    kf = KFold(K)
    contador = 0

    for train, test in kf.split(df):
        train_df = df.iloc[train].copy()
        test_df  = df.iloc[test].copy()

        train_df.reset_index(drop=True, inplace=True)
        test_df.reset_index(drop=True, inplace=True)

        A_train = crear_matriz_A(fitting_cat, train_df)
        y_train = train_df['metroscubiertos']
        
        A_test = crear_matriz_A(fitting_cat, test_df)
        y_test = test_df['metroscubiertos']

        regresor = metnum.LinearRegression()
        regresor.fit(A_train, y_train)
        pred = regresor.predict(A_test)
                
        indices = np.where(pred >= 0)[0]
              
        err = mean_squared_log_error(pred[indices], y_test[indices])
        
        errores.append(err)

    return errores



def cross_val(df, K, fitting_cat, testing_cat = []):
    errores = []
    
    kf = KFold(K)
    contador = 0

    for train, test in kf.split(df):
        train_df = df.iloc[train].copy()
        test_df  = df.iloc[test].copy()

        train_df.reset_index(drop=True, inplace=True)
        test_df.reset_index(drop=True, inplace=True)

        A_train = crear_matriz_A(fitting_cat, train_df)
        y_train = train_df['metroscubiertos']
        
        A_test = crear_matriz_A(fitting_cat, test_df)
        y_test = test_df['metroscubiertos']

        regresor = metnum.LinearRegression()
        regresor.fit(A_train, y_train)
        pred = regresor.predict(A_test)
        
        if testing_cat == []:
            err = mean_squared_error(pred, y_test)
            
        else:
            y_test = y_test.to_numpy()
            indices = []
            for i in range(len(test_df)):
                if hay_que_testear(test_df, i, testing_cat):
                    contador += 1
                    indices.append(i)
            
            pred = pred[indices]
            y_test = y_test[indices]
            err = mean_squared_error(pred, y_test)
        
        errores.append(err)
        
    if contador != 0:
        print(contador)
    return errores

def hay_que_testear(df, i, categories):
    for cat in categories:
        if test_df.iloc[i]['tipodepropiedad'] == cat:
            return True
    return False

def agregar_pbi(df):
    pbi_prov=np.array([21079,6199,9239,12502,16228,5890,8118,8011,6961,7235,11381,8833,12838,9449,11343,8044,4586,6508,5522,9347,7888,8108,9975,11060,6220,3592,9177,14364,5113,4446,51460,7600])
    
    df_copy = df.copy()
    
    df_copy.dropna(subset= ['provincia'], inplace=True)
    df_copy.reset_index(drop=True, inplace=True)
    
    df_prov = df_copy['provincia'].value_counts().rename_axis('provincia').reset_index(name='counts')
    del df_prov['counts']
    df_prov['PBI']=pd.Series(pbi_prov)
    df_prov.set_index('provincia', inplace=True)


    pbi_por_prov=np.array([], dtype=np.single)
    for i in range (len(df_copy)):
        prov=df_copy.iloc[i]['provincia']
        pbi_por_prov=np.append(pbi_por_prov,float(df_prov.loc[prov]['PBI']))


    df_copy['PBI']=pd.Series(pbi_por_prov)
    
    return df_copy

def agregar_flia(df):
    
    df_copy = df.copy()
    
    df_copy.dropna(subset= ['banos', 'piscina', 'habitaciones', 'escuelascercanas'], inplace=True)
    df_copy.reset_index(drop=True, inplace=True)
    df_copy['bueno para familias'] = ((df_copy['banos']>1) & (df_copy['piscina']>0) & (df_copy['habitaciones']>2) & (df_copy['escuelascercanas']>0)).astype(int)
    
    return df_copy

def agregar_vacacional(df):
    df_copy = df.copy()
    
    filtro_provincias = (df_copy['provincia'] == 'Quintana Roo') | (df_copy['provincia'] == 'Campeche')
    filtro_ciudades = (df_copy['ciudad'] == 'Veracruz') | (df_copy['ciudad'] == 'Boca del Río') | (df_copy['ciudad'] == 'Alvarado') | (df_copy['ciudad'] == 'Los Cabos') | (df_copy['ciudad'] == 'La Paz') | (df_copy['ciudad'] == 'Puerto Vallarta') | (df_copy['ciudad'] == 'Mazatlán') | (df_copy['ciudad'] == 'Colima')

    df_copy['es_costero'] = (filtro_provincias | filtro_ciudades | (df_copy['piscina'] > 0)).astype(int)
    
    return df_copy


def contar_apariciones(listas, categorias):
    apariciones = {}
    for cat in categorias:
        apariciones[cat] = 0
    
    for lista in listas:
        for elem in lista:
            apariciones[elem] += 1
    return apariciones

def find_region(provincia):
    regiones_dicc = {
        'noroeste': np.array(['Baja California Sur', 'Baja California Norte']),
       'pacifico' : np.array(['Sonora','Sinaloa']),
        'norte'   : np.array(['Chihuahua', 'Coahuila', 'Nuevo León', 'Tamaulipas', 'Durango']),
       'occidente': np.array(['Nayarit', 'Jalisco', 'Colima', 'Michoacán']),
       'distrito' : np.array(['Distrito Federal', 'Hidalgo', 'Edo. de México']),
        'bajio'   : np.array(['Aguascalientes', 'Guanajuato', 'Querétaro', 'San luis Potosí', 'Zacatecas']),
        'golfo'   : np.array(['Morelos', 'Oaxaca', 'Puebla', 'Veracruz', 'Tabasco', 'Tlaxcala', 'Guerrero']),
       'suroeste' : np.array(['Yucatán', 'Campeche', 'Quintana Roo', 'Chiapas'])
    }
    
    for key in regiones_dicc:
        if provincia in regiones_dicc[key]:
            return key

        
def agregar_region(_df):
    df = _df.copy()
    df['region'] = pd.Series(1)
    
    for i in tqdm(range(len(df))):
        provincia = df.iloc[i]['provincia']
        df.loc[i, 'region'] = find_region(provincia)
    
    return df


def sort_dicc(d):
    return {k: v for k, v in sorted(d.items(), key=lambda item: item[1])}

In [6]:
def cross_val_graph(df, K, fitting_cat1, fitting_cat2):
    pred1 = []
    pred2 = []
    reales = []
    
    kf = KFold(K)

    for train, test in kf.split(df):
        train_df = df.iloc[train].copy()
        test_df  = df.iloc[test].copy()

        train_df.reset_index(drop=True, inplace=True)
        test_df.reset_index(drop=True, inplace=True)

        A_train1 = crear_matriz_A(fitting_cat1, train_df)       
        A_train2 = crear_matriz_A(fitting_cat2, train_df)
        y_train = train_df['metroscubiertos']
        
        
        A_test1 = crear_matriz_A(fitting_cat1, test_df)
        A_test2 = crear_matriz_A(fitting_cat2, test_df)
        y_test = test_df['metroscubiertos']

        regresor1 = metnum.LinearRegression()
        regresor1.fit(A_train1, y_train)
        _pred1 = regresor1.predict(A_test1)
        
        regresor2 = metnum.LinearRegression()
        regresor2.fit(A_train2, y_train)
        _pred2 = regresor2.predict(A_test2)

        
        indices = np.where((_pred1 >= 0) & (_pred1 >= 0))[0]
        
        
        pred1.append(_pred1[indices])
        pred2.append(_pred2[indices])
        reales.append(y_test[indices])
        
    return (pred1, pred2, reales)



def cross_val_provincias(df, K, fitting_cat):
        
    matrices_A_train = {}

    regresores = {}
    
    y_trains = {}
    
    errores = []
    
    kf = KFold(K)

    for train, test in kf.split(df):
        print('Entre a kfold')
        train_df = df.iloc[train].copy()
        test_df  = df.iloc[test].copy()

        train_df.reset_index(drop=True, inplace=True)
        test_df.reset_index(drop=True, inplace=True)

        
        for region in regiones:
            region_train_df = train_df[train_df['region'] == region]
            matrices_A_train[region] = crear_matriz_A(fitting_cat, region_train_df)
            y_trains[region] = region_train_df['metroscubiertos']
            
            regresores[region] = metnum.LinearRegression()
            regresores[region].fit(matrices_A_train[region], y_trains[region])
            
            
        preds = []
        reales = []
        
        for i in range(len(test_df)):
            dato_df = test_df[test_df.index == i]
            A_test = crear_matriz_A(fitting_cat, dato_df)
            y_test = dato_df['metroscubiertos']
            reg_test = dato_df['region'].to_numpy()[0]
            preds.append(regresores[reg_test].predict(A_test)[0,0])
            reales.append(float(y_test))

        preds = np.array(preds)
        reales = np.array(reales)
        
        err = mean_squared_error(preds, reales)
            
        errores.append(err)
        
    return errores


def cross_val_log_provincias(df, K, fitting_cat):
        
    matrices_A_train = {}

    regresores = {}
    
    y_trains = {}
    
    errores = []
    
    kf = KFold(K)

    for train, test in kf.split(df):
        print('Entre a kfold')
        train_df = df.iloc[train].copy()
        test_df  = df.iloc[test].copy()

        train_df.reset_index(drop=True, inplace=True)
        test_df.reset_index(drop=True, inplace=True)

        
        for region in regiones:
            region_train_df = train_df[train_df['region'] == region]
            matrices_A_train[region] = crear_matriz_A(fitting_cat, region_train_df)
            y_trains[region] = region_train_df['metroscubiertos']
            
            regresores[region] = metnum.LinearRegression()
            regresores[region].fit(matrices_A_train[region], y_trains[region])
            
            
        preds = []
        reales = []
        
        for i in range(len(test_df)):
            dato_df = test_df[test_df.index == i]
            A_test = crear_matriz_A(fitting_cat, dato_df)
            y_test = dato_df['metroscubiertos']
            reg_test = dato_df['region'].to_numpy()[0]
            preds.append(regresores[reg_test].predict(A_test)[0,0])
            reales.append(float(y_test))

        preds = np.array(preds)
        reales = np.array(reales)
        
        indices = np.where(preds >= 0)[0]
        
        err = mean_squared_log_error(preds[indices], reales[indices])
            
        errores.append(err)
        
    return errores

In [65]:
def agregar_propiedad(_df):
    losquesi = np.array(['Casa', 'Apartamento', 'Casa en condominio'])
    df = _df.copy()
    
    for i in tqdm(range(len(df))):
        prop = df.iloc[i]['tipodepropiedad']
        if not (prop in losquesi):
            df.loc[i, 'tipodepropiedad'] = 'Otro'
            
    return df
    

def cross_val_propiedades(df, K, fitting_cat):
        
    matrices_A_train = {}

    regresores = {}
    
    y_trains = {}
    
    errores = []
    
    kf = KFold(K)

    for train, test in kf.split(df):
        print('Entre a kfold')
        train_df = df.iloc[train].copy()
        test_df  = df.iloc[test].copy()

        train_df.reset_index(drop=True, inplace=True)
        test_df.reset_index(drop=True, inplace=True)

        
        for propiedad in propiedades:
            propiedad_train_df = train_df[train_df['tipodepropiedad'] == propiedad]
            matrices_A_train[propiedad] = crear_matriz_A(fitting_cat, propiedad_train_df)
            y_trains[propiedad] = propiedad_train_df['metroscubiertos']
            
            regresores[propiedad] = metnum.LinearRegression()
            regresores[propiedad].fit(matrices_A_train[propiedad], y_trains[propiedad])
            
            
        preds = []
        reales = []
        
        for i in range(len(test_df)):
            dato_df = test_df[test_df.index == i]
            A_test = crear_matriz_A(fitting_cat, dato_df)
            y_test = dato_df['metroscubiertos']
            prop_test = dato_df['tipodepropiedad'].to_numpy()[0]
            preds.append(regresores[prop_test].predict(A_test)[0,0])
            reales.append(float(y_test))

        preds = np.array(preds)
        reales = np.array(reales)
        
        err = mean_squared_error(preds, reales)
            
        errores.append(err)
        
    return errores


def cross_val_log_propiedades(df, K, fitting_cat):
    
    matrices_A_train = {}

    regresores = {}
    
    y_trains = {}
    
    errores = []
    
    kf = KFold(K)

    for train, test in kf.split(df):
        print('Entre a kfold')
        train_df = df.iloc[train].copy()
        test_df  = df.iloc[test].copy()

        train_df.reset_index(drop=True, inplace=True)
        test_df.reset_index(drop=True, inplace=True)

        
        for propiedad in propiedades:
            propiedad_train_df = train_df[train_df['tipodepropiedad'] == propiedad]
            matrices_A_train[propiedad] = crear_matriz_A(fitting_cat, propiedad_train_df)
            y_trains[propiedad] = propiedad_train_df['metroscubiertos']
            
            regresores[propiedad] = metnum.LinearRegression()
            regresores[propiedad].fit(matrices_A_train[propiedad], y_trains[propiedad])
            
            
        preds = []
        reales = []
        
        for i in range(len(test_df)):
            dato_df = test_df[test_df.index == i]
            A_test = crear_matriz_A(fitting_cat, dato_df)
            y_test = dato_df['metroscubiertos']
            prop_test = dato_df['tipodepropiedad'].to_numpy()[0]
            preds.append(regresores[prop_test].predict(A_test)[0,0])
            reales.append(float(y_test))

        preds = np.array(preds)
        reales = np.array(reales)
        
        indices = np.where(preds >= 0)[0]
        
        err = mean_squared_log_error(preds[indices], reales[indices])
            
        errores.append(err)
        
    return errores

In [86]:
def cross_val_final(df, K, fitting_cat):
    
    matrices_A_train = {}

    regresores = {}
    
    y_trains = {}
    
    errores = []
    
    kf = KFold(K)

    for train, test in kf.split(df):
        train_df = df.iloc[train].copy()
        test_df  = df.iloc[test].copy()

        train_df.reset_index(drop=True, inplace=True)
        test_df.reset_index(drop=True, inplace=True)

        
        for region in regiones:
                regresores[region] = {}
                matrices_A_train[region] = {}
                y_trains[region] = {}
                
                for propiedad in propiedades:
                    propiedad_train_df = train_df[(train_df['tipodepropiedad'] == propiedad) & (train_df['region'] == region)]
                    matrices_A_train[region][propiedad] = crear_matriz_A(fitting_cat, propiedad_train_df)
                    y_trains[region][propiedad] = propiedad_train_df['metroscubiertos']

                    regresores[region][propiedad] = metnum.LinearRegression()
                    regresores[region][propiedad].fit(matrices_A_train[region][propiedad], y_trains[region][propiedad])
            
        preds = []
        reales = []
        
        for i in range(len(test_df)):
            dato_df = test_df[test_df.index == i]
            A_test = crear_matriz_A(fitting_cat, dato_df)
            y_test = dato_df['metroscubiertos']
            reg_test = dato_df['region'].to_numpy()[0]
            prop_test = dato_df['tipodepropiedad'].to_numpy()[0]
            preds.append(regresores[reg_test][prop_test].predict(A_test)[0,0])
                               
            reales.append(float(y_test))

        preds = np.array(preds)
        reales = np.array(reales)
        
        err = mean_squared_error(preds, reales)
            
        errores.append(err)
        
    return errores


def cross_val_log_final(df, K, fitting_cat):
    
    matrices_A_train = {}

    regresores = {}
    
    y_trains = {}
    
    errores = []
    
    kf = KFold(K)

    for train, test in kf.split(df):
        train_df = df.iloc[train].copy()
        test_df  = df.iloc[test].copy()

        train_df.reset_index(drop=True, inplace=True)
        test_df.reset_index(drop=True, inplace=True)

        
        for region in regiones:
                regresores[region] = {}
                matrices_A_train[region] = {}
                y_trains[region] = {}
                
                for propiedad in propiedades:
                    propiedad_train_df = train_df[(train_df['tipodepropiedad'] == propiedad) & (train_df['region'] == region)]
                    matrices_A_train[region][propiedad] = crear_matriz_A(fitting_cat, propiedad_train_df)
                    y_trains[region][propiedad] = propiedad_train_df['metroscubiertos']

                    regresores[region][propiedad] = metnum.LinearRegression()
                    regresores[region][propiedad].fit(matrices_A_train[region][propiedad], y_trains[region][propiedad])
            
        preds = []
        reales = []
        
        for i in range(len(test_df)):
            dato_df = test_df[test_df.index == i]
            A_test = crear_matriz_A(fitting_cat, dato_df)
            y_test = dato_df['metroscubiertos']
            reg_test = dato_df['region'].to_numpy()[0]
            prop_test = dato_df['tipodepropiedad'].to_numpy()[0]
            preds.append(regresores[reg_test][prop_test].predict(A_test)[0,0])
                               
            reales.append(float(y_test))

        preds = np.array(preds)
        reales = np.array(reales)
        
        indices = np.where(preds >= 0)[0]
        
        err = mean_squared_log_error(preds[indices], reales[indices])
            
        errores.append(err)
        
    return errores

In [9]:
cat_11 = np.array(['metrostotales','precio','banos','habitaciones','garages','gimnasio','usosmultiples','piscina','escuelascercanas','centroscomercialescercanos', 'antiguedad'])
cat_simples = np.array(['precio', 'banos', 'habitaciones', 'gimnasio'])
cat_pbi = np.array(['precio', 'banos', 'habitaciones', 'gimnasio', 'PBI'])

propiedades = np.array(['Casa','Apartamento', 'Casa en condominio', 'Otro']) 
regiones = np.array(['noroeste','pacifico','norte','occidente','distrito','bajio','golfo','suroeste'])

### Experimento 1

#### Hiperplano basico

In [26]:
df_base = df.copy()
df_base.dropna(subset= ['metroscubiertos'], inplace=True)
df_base.reset_index(drop=True, inplace=True)
df_base

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,23533.0,NaN,NaN,2015-08-23 00:00:00,0.0,0.0,0.0,0.0,0.0,2273000.0
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,24514.0,19.310205,-99.227655,2013-06-28 00:00:00,0.0,0.0,0.0,1.0,1.0,3600000.0
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,48551.0,NaN,NaN,2015-10-17 00:00:00,0.0,0.0,0.0,0.0,0.0,1200000.0
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,53666.0,19.301890,-99.688015,2012-03-09 00:00:00,0.0,0.0,0.0,1.0,1.0,650000.0
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,47835.0,NaN,NaN,2016-06-07 00:00:00,0.0,0.0,0.0,0.0,0.0,1150000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222595,119879,bonita casas de 2 recamaras a 10 minutos del c...,vendo casa en bosques de ica residencial a 10 ...,Casa,BOSQUES,Zinacantepec,Edo. de México,0.0,2.0,2.0,...,53666.0,NaN,NaN,2015-02-08 00:00:00,0.0,0.0,0.0,0.0,0.0,650000.0
222596,259178,casa en condominio a 10 min. del centro de toluca,"casa con un jardin amplio, un cuarto de servic...",Casa,Filiberto Navas 325,Toluca,Edo. de México,0.0,3.0,3.0,...,51954.0,19.294665,-99.692916,2014-07-10 00:00:00,0.0,0.0,0.0,1.0,1.0,1940000.0
222597,131932,nicolas san juan,"departamento con excelente ubicación, muy cerc...",Apartamento,Nicolas San Juan,Benito Juárez,Distrito Federal,20.0,2.0,1.0,...,50003995.0,NaN,NaN,2015-03-03 00:00:00,0.0,0.0,0.0,0.0,0.0,3400000.0
222598,146867,casa sola. javier rojo gomez.,"casa sola, dividida en cuatro departamentos de...",Casa,Javier Rojo Gomez 120,Iztapalapa,Distrito Federal,20.0,4.0,0.0,...,24162.0,19.366651,-99.082246,2014-12-26 00:00:00,1.0,0.0,0.0,1.0,1.0,2890000.0


In [76]:
cat_todos = df.columns
cat_9 = np.array(['metrostotales','precio','banos','habitaciones','garages','gimnasio','usosmultiples','piscina', 'antiguedad'])

categorias1 = np.array(['metrostotales','precio','banos','habitaciones','garages','gimnasio','usosmultiples','piscina', 'antiguedad'])
df1 = limpiar(df_base, categorias1)
print(f'El dataset quedo con: {len(df1)}')
errores = cross_val(df1, 5, categorias1)
print(f'Todas las categorias:')
print(f'std: {np.sqrt(np.std(errores))} mean: {np.sqrt(np.mean(errores))}')


categorias1 = np.array(['metrostotales'])
df1 = limpiar(df_base, categorias1)
print(f'\n\nEl dataset quedo con: {len(df1)}')
errores = cross_val(df1, 5, categorias1)
print(f'Solo metros totales:')
print(f'std: {np.sqrt(np.std(errores))} mean: {np.sqrt(np.mean(errores))}')


categorias1 = np.array(['precio'])
df1 = limpiar(df_base, categorias1)
print(f'\n\nEl dataset quedo con: {len(df1)}')
errores = cross_val(df1, 5, categorias1)
print(f'Solo precio:')
print(f'std: {np.sqrt(np.std(errores))} mean: {np.sqrt(np.mean(errores))}')


categorias1 = np.array(['metrostotales','precio'])
df1 = limpiar(df_base, categorias1)
print(f'\n\nEl dataset quedo con: {len(df1)}')
errores = cross_val(df1, 5, categorias1)
print(f'Metros totales y precio:')
print(f'std: {np.sqrt(np.std(errores))} mean: {np.sqrt(np.mean(errores))}')

El dataset quedo con: 120510
Todas las categorias:
std: 30.414866796994776 mean: 2163


El dataset quedo con: 171133
Solo metros cubiertos:
std: 32.252310638260674 mean: 4429


El dataset quedo con: 222600
Solo cosas malas:
std: 76.99851748751807 mean: 9898


El dataset quedo con: 171133
2 cosas buenas:
std: 25.931763749731523 mean: 3554


### Experimento 2

#### Todas las combinaciones posibles

In [41]:
iterable = np.array(['metrostotales','precio','banos','habitaciones','garages','gimnasio','usosmultiples','piscina','escuelascercanas','centroscomercialescercanos', 'antiguedad'])

df_exp2 = limpiar(df_base, cat_todos)

N_exp2 = len(df_exp2)
N_train2 = round(N_exp2 * 0.8)

exp2_train = df_exp2[:N_train2].copy()
exp2_val   = df_exp2[N_train2:].copy()

exp2_train.reset_index(drop=True, inplace=True)
exp2_val.reset_index(drop=True, inplace=True)

y_exp2_train = exp2_train['metroscubiertos']
y_exp2_val = exp2_val['metroscubiertos']

regresor_exp2 = metnum.LinearRegression()


combinaciones = []
for r in range(1, len(iterable)):
    combinaciones.append(list(itertools.combinations(iterable, r)))

    
winner_error_exp2 = -1
winner_list_exp2 = []
errores_exp2 = []
listas_exp2 = []
for combinacion in tqdm(combinaciones):
    for tupla in combinacion:
        if len(tupla) <= 4:
            A_exp2_train = crear_matriz_A(np.array(tupla), exp2_train)
            regresor_exp2.fit(A_exp2_train, y_exp2_train)

            A_exp2_val = crear_matriz_A(np.array(tupla), exp2_val)
            pred_exp2 = regresor_exp2.predict(A_exp2_val)

            current_error = mean_squared_error(pred_exp2, y_exp2_val)

            errores_exp2.append(current_error)
            listas_exp2.append(tupla)
            if(winner_error_exp2 == -1 or winner_error_exp2 > current_error):
                winner_error_exp2 = current_error
                winner_list_exp2 = list(tupla)


indices_exp2 = np.argsort(errores_exp2)
sorted_errores_exp2 = np.array(errores_exp2)[indices_exp2]
sorted_lists_exp2 = np.array(listas_exp2)[indices_exp2]

for e, l in zip(sorted_errores_exp2, sorted_lists_exp2):
    print(f'Lista: {l}, Error: {e}')

100%|██████████| 10/10 [00:58<00:00,  5.80s/it]

Lista: ('metrostotales', 'precio', 'banos', 'habitaciones'), Error: 2254.1248733906505
Lista: ('metrostotales', 'precio', 'banos', 'antiguedad'), Error: 2278.689975187455
Lista: ('metrostotales', 'precio', 'banos', 'piscina'), Error: 2314.4298200974717
Lista: ('metrostotales', 'precio', 'banos', 'gimnasio'), Error: 2322.299045992025
Lista: ('metrostotales', 'precio', 'banos', 'usosmultiples'), Error: 2335.196229731741
Lista: ('metrostotales', 'precio', 'banos', 'garages'), Error: 2342.330036836692
Lista: ('metrostotales', 'precio', 'banos', 'centroscomercialescercanos'), Error: 2343.228302679776
Lista: ('metrostotales', 'precio', 'banos', 'escuelascercanas'), Error: 2347.6868827497965
Lista: ('metrostotales', 'precio', 'banos'), Error: 2348.5918961266893
Lista: ('metrostotales', 'precio', 'habitaciones', 'centroscomercialescercanos'), Error: 2387.204695597833
Lista: ('metrostotales', 'precio', 'habitaciones', 'gimnasio'), Error: 2396.19438590984
Lista: ('metrostotales', 'precio', 'habi

##### Contamos la cantidad de apariciones

In [42]:
dicc_apariciones = contar_apariciones(sorted_lists_exp2[:50], iterable)

print('\n\nApariciones de cada caracteristica en las 50 mejores combinaciones')
dicc_apariciones = sort_dicc(dicc_apariciones)
for key in dicc_apariciones:
    print(f'{key}: {dicc_apariciones[key]}')



Apariciones de cada caracteristica en las 50 mejores combinaciones
gimnasio: 9
usosmultiples: 9
escuelascercanas: 9
garages: 10
piscina: 10
centroscomercialescercanos: 10
antiguedad: 10
precio: 17
habitaciones: 17
banos: 42
metrostotales: 50


##### Errores de la categoria simple

In [82]:
cat_simples = np.array(['metrostotales', 'banos', 'habitaciones', 'precio'])
df_simples = limpiar(df_base, cat_todos)
errores = cross_val(df_simples, 5, cat_simples)
print(f'El dataset quedo con: {len(df_simples)}')
print(f'Mejor combinacion 4 categorias:')
print(cat_simples)
print(f'std: {np.sqrt(np.std(errores))} mean: {np.sqrt(np.mean(errores))}')


cat_simples = np.array(['metrostotales', 'banos', 'habitaciones', 'precio'])
df_simples = limpiar(df_base, cat_todos)
errores = cross_val_log(df_simples, 5, cat_simples)
print(f'El dataset quedo con: {len(df_simples)}')
print(f'Mejor combinacion 4 categorias (logaritmico):')
print(cat_simples)
print(f'std: {np.sqrt(np.std(errores))} mean: {np.sqrt(np.mean(errores))}')

El dataset quedo con: 49881
Mejor combinacion 4 categorias:
['metrostotales' 'banos' 'habitaciones' 'precio']
std: 27.611662007487734 mean: 47.09909072912682
El dataset quedo con: 49881
Mejor combinacion 4 categorias:
['metrostotales' 'banos' 'habitaciones' 'precio']
std: 0.0006744481435038399 mean: 0.29297320217398787


##### Error de la categoria compleja

In [83]:
df_complejo = limpiar(df_base, cat_todos)
errores = cross_val(df_complejo, 5, cat_9)
print(f'El dataset quedo con: {len(df_complejo)}')
print(f'9 categorias numericas:')
print(cat_9)
print(f'std: {np.sqrt(np.std(errores))} mean: {np.sqrt(np.mean(errores))}')


errores = cross_val_log(df_complejo, 5, cat_9)
print(f'El dataset quedo con: {len(df_complejo)}')
print(f'9 categorias numericas (logaritmico):')
print(cat_9)
print(f'std: {np.sqrt(np.std(errores))} mean: {np.sqrt(np.mean(errores))}')

El dataset quedo con: 49881
Mejor combinacion 9 categorias:
['metrostotales' 'precio' 'banos' 'habitaciones' 'garages' 'gimnasio'
 'usosmultiples' 'piscina' 'antiguedad']
std: 28.068369545418143 mean: 46.627833213406866
El dataset quedo con: 49881
Mejor combinacion 9 categorias:
['metrostotales' 'precio' 'banos' 'habitaciones' 'garages' 'gimnasio'
 'usosmultiples' 'piscina' 'antiguedad']
std: 0.0005784396109845735 mean: 0.29189739119417424


### Experimento 3

#### Segmentar por regiones

##### Separamos por regiones

In [49]:
regiones = np.array(['noroeste','pacifico','norte','occidente','distrito','bajio','golfo','suroeste'])

noroeste = np.array(['Baja California Sur', 'Baja California Norte'])
pacifico = np.array(['Sonora','Sinaloa'])
norte    = np.array(['Chihuahua', 'Coahuila', 'Nuevo León', 'Tamaulipas', 'Durango'])
occidente= np.array(['Nayarit', 'Jalisco', 'Colima', 'Michoacán'])
distrito = np.array(['Distrito Federal', 'Hidalgo', 'Edo. de México'])
bajio    = np.array(['Aguascalientes', 'Guanajuato', 'Querétaro', 'San luis Potosí', 'Zacatecas'])
golfo    = np.array(['Morelos', 'Oaxaca', 'Puebla', 'Veracruz', 'Tabasco', 'Tlaxcala', 'Guerrero'])
suroeste = np.array(['Yucatán', 'Campeche', 'Quintana Roo', 'Chiapas'])

##### Calculamos el error por region


In [50]:
df_seg_prov = limpiar(df_base, cat_todos)
df_seg_prov = agregar_region(df_seg_prov)

100%|██████████| 49881/49881 [00:56<00:00, 887.11it/s] 


In [79]:
errores_cuad = {}
errores_log = {}

for region in regiones:
    df_region = df_seg_prov[df_seg_prov['region'] == region].copy()
    errores = cross_val(df_region, 5, cat_simples)
    errores_cuad[region] = np.sqrt(np.mean(errores))
    errores = cross_val_log(df_region, 5, cat_simples)
    errores_log[region] = np.sqrt(np.mean(errores))
    
errores_cuad = sort_dicc(errores_cuad)
errores_log = sort_dicc(errores_log)

for key in errores_cuad:
    print(f'{key}: {errores_cuad[key]}')
    
print('\n\n')
for key in errores_log:
    print(f'{key}: {errores_log[key]:.4}')

noroeste: 28.856471662134986
bajio: 38.269662785903904
pacifico: 38.991423855931835
occidente: 39.17922307898421
suroeste: 43.660961006061655
norte: 46.265726397254745
golfo: 47.159552129603014
distrito: 49.70276057394746



bajio: 0.2227
occidente: 0.2512
noroeste: 0.2521
pacifico: 0.2579
suroeste: 0.2627
golfo: 0.2744
norte: 0.2894
distrito: 0.3042


##### Ahora el error total, pero con los regresores segmentados

In [80]:
errores = cross_val_provincias(df_seg_prov, 5, cat_simples)

print(f'Dataset segmentado por region:')
print(f'El dataset quedo con: {len(df_seg_prov)}')
print(f'Entrenando con las 4 mejores categorias:')
print(cat_simples)
print(f'std: {np.sqrt(np.std(errores))} mean: {np.sqrt(np.mean(errores))}')

errores = cross_val_log_provincias(df_seg_prov, 5, cat_simples)

print(f'\n\nDataset segmentado por region (logaritmico):')
print(f'El dataset quedo con: {len(df_seg_prov)}')
print(f'Entrenando con las 4 mejores categorias:')
print(cat_simples)
print(f'std: {np.sqrt(np.std(errores))} mean: {np.sqrt(np.mean(errores))}')

Entre a kfold
Entre a kfold
Entre a kfold
Entre a kfold
Entre a kfold
Dataset segmentado:
El dataset quedo con: 49881
Entrenando con las 4 mejores:
['metrostotales' 'banos' 'habitaciones' 'precio']
std: 27.658028597736312 mean: 45.64138233575789
Entre a kfold
Entre a kfold
Entre a kfold
Entre a kfold
Entre a kfold


Dataset segmentado log (todo menos df):
El dataset quedo con: 49881
Entrenando con las 4 mejores:
['metrostotales' 'banos' 'habitaciones' 'precio']
std: 0.0005704778841614536 mean: 0.28064423738751676


### Experimento 4

#### Segmentar por propiedad en todo el dataset

In [62]:
df_seg_vivienda = limpiar(df_base, cat_todos)
df_seg_vivienda = agregar_propiedad(df_seg_vivienda)

100%|██████████| 49881/49881 [00:07<00:00, 6521.37it/s]


In [81]:
errores = cross_val_propiedades(df_seg_vivienda, 5, cat_simples)

print(f'Dataset segmentado por vivienda:')
print(f'El dataset quedo con: {len(df_seg_vivienda)}')
print(f'Entrenando con las 4 mejores categorias:')
print(cat_simples)
print(f'std: {np.sqrt(np.std(errores))} mean: {np.sqrt(np.mean(errores))}')

errores = cross_val_log_propiedades(df_seg_vivienda, 5, cat_simples)

print(f'\n\nDataset segmentado por vivienda (logaritmico):')
print(f'El dataset quedo con: {len(df_seg_vivienda)}')
print(f'Entrenando con las 4 mejores categorias:')
print(cat_simples)
print(f'std: {np.sqrt(np.std(errores))} mean: {np.sqrt(np.mean(errores)):.4}')


Entre a kfold
Entre a kfold
Entre a kfold
Entre a kfold
Entre a kfold
Dataset segmentado por vivienda:
El dataset quedo con: 49881
Entrenando con las 4 mejores:
['metrostotales' 'banos' 'habitaciones' 'precio']
std: 21.49838670580629 mean: 42.173003176123316
Entre a kfold
Entre a kfold
Entre a kfold
Entre a kfold
Entre a kfold


Dataset segmentado log (todo menos df):
El dataset quedo con: 49881
Entrenando con las 4 mejores:
['metrostotales' 'banos' 'habitaciones' 'precio']
std: 0.000703895708482747 mean: 0.2495


##### Calculamos el error por propiedad

In [85]:
errores_cuad = {}
errores_log = {}

for propiedad in propiedades:
    df_propiedad = df_seg_vivienda[df_seg_vivienda['tipodepropiedad'] == propiedad].copy()
    errores = cross_val(df_propiedad, 5, cat_simples)
    errores_cuad[propiedad] = np.sqrt(np.mean(errores))
    errores = cross_val_log(df_propiedad, 5, cat_simples)
    errores_log[propiedad] = np.sqrt(np.mean(errores))
    
errores_cuad = sort_dicc(errores_cuad)
errores_log = sort_dicc(errores_log)

for key in errores_cuad:
    print(f'{key}: {errores_cuad[key]}')
    
print('\n\n')
for key in errores_log:
    print(f'{key}: {errores_log[key]:.4}')

Apartamento: 19.68894983773724
Casa en condominio: 41.435430719565076
Casa: 47.13619743625172
Otro: 69.0384766059532



Apartamento: 0.1448
Casa en condominio: 0.2319
Casa: 0.2755
Otro: 0.4249


### Ahora metemos esa segmentacion en el dataset completo

In [89]:
df_final = df_seg_prov.copy()
df_final = agregar_propiedad(df_final)

100%|██████████| 49881/49881 [00:07<00:00, 6369.70it/s]


In [91]:
errores = cross_val_final(df_final, 5, cat_simples)

print(f'Dataset segmentado por regiones y por propiedad:')
print(f'El dataset quedo con: {len(df_final)}')
print(f'Entrenando con las 4 mejores categorias:')
print(cat_simples)
print(f'std: {np.sqrt(np.std(errores))} mean: {np.sqrt(np.mean(errores))}')

errores = cross_val_log_final(df_final, 5, cat_simples)

print(f'\n\nDataset segmentado por regiones y por propiedad (logaritimico):')
print(f'El dataset quedo con: {len(df_final)}')
print(f'Entrenando con las 4 mejores categorias:')
print(cat_simples)
print(f'std: {np.sqrt(np.std(errores))} mean: {np.sqrt(np.mean(errores))}')

Dataset segmentado por regiones y por propiedad:
El dataset quedo con: 49881
Entrenando con las 4 mejores:
['metrostotales' 'banos' 'habitaciones' 'precio']
std: 4.905073694378103 mean: 41.06859733918846


Dataset segmentado por regiones y por propiedad:
El dataset quedo con: 49881
Entrenando con las 4 mejores:
['metrostotales' 'banos' 'habitaciones' 'precio']
std: 0.029811863592835962 mean: 0.2410273887251441


### Experimento 5

#### Feature engineering: PBI

In [105]:
df_pbi = limpiar(df_base, cat_todos)
df_pbi = agregar_pbi(df_pbi)
df_pbi = agregar_propiedad(df_pbi)
df_pbi = agregar_region(df_pbi)

100%|██████████| 49881/49881 [00:35<00:00, 1417.45it/s]


In [110]:
cat_pbi = np.array(['metrostotales','precio', 'banos', 'habitaciones', 'PBI'])

errores = cross_val_final(df_pbi, 5, cat_pbi)

print(f'Dataset doble segmentacion, agregamos PBI:')
print(f'El dataset quedo con: {len(df_pbi)}')
print(f'Entrenando con las 4 mejores categorias + PBI:')
print(cat_pbi)
print(f'std: {np.sqrt(np.std(errores))} mean: {np.sqrt(np.mean(errores))}')

errores = cross_val_log_final(df_pbi, 5, cat_pbi)

print(f'Dataset doble segmentacion, agregamos PBI (logaritmico):')
print(f'El dataset quedo con: {len(df_pbi)}')
print(f'Entrenando con las 4 mejores categorias + PBI:')
print(cat_pbi)
print(f'std: {np.sqrt(np.std(errores))} mean: {np.sqrt(np.mean(errores))}')

Dataset completo, agregamos PBI:
El dataset quedo con: 49881
Entrenando con las 4 mejores + PBI:
['metrostotales' 'precio' 'banos' 'habitaciones' 'PBI']
std: 5.001136821899387 mean: 40.97253234447981
Dataset completo, agregamos PBI:
El dataset quedo con: 49881
Entrenando con las 4 mejores + PBI:
['metrostotales' 'precio' 'banos' 'habitaciones' 'PBI']
std: 0.03021564760526951 mean: 0.23982656732561877


### Experimento 6

#### Feature engineering: Bueno para la familia

In [106]:
df_flia = df_pbi.copy()
df_flia = agregar_flia(df_flia)

In [109]:
cat_flia = np.array(['metrostotales','precio', 'banos', 'habitaciones', 'bueno para familias'])
df_flia = limpiar(df, cat_11)
df_flia = agregar_pbi(df_flia)
df_flia = agregar_flia(df_flia)
errores = cross_val_final(df_flia, 5, cat_flia)

print(f'Dataset doble segmentacion, agregamos FLIA:')
print(f'El dataset quedo con: {len(df_flia)}')
print(f'Entrenando con las 4 mejores + FLIA:')
print(cat_flia)
print(f'std: {np.sqrt(np.std(errores))} mean: {np.sqrt(np.mean(errores))}')


errores = cross_val_log_final(df_flia, 5, cat_flia)

print(f'Dataset doble segmentacion, agregamos FLIA (logaritmico):')
print(f'El dataset quedo con: {len(df_flia)}')
print(f'Entrenando con las 4 mejores + FLIA:')
print(cat_flia)
print(f'std: {np.sqrt(np.std(errores))} mean: {np.sqrt(np.mean(errores))}')

Dataset completo, agregamos FLIA:
El dataset quedo con: 49881
Entrenando con las 4 mejores + PBI + FLIA:
['metrostotales' 'precio' 'banos' 'habitaciones' 'bueno para familias']
std: 25.14566441012622 mean: 41.01647351511851
Dataset completo, agregamos FLIA:
El dataset quedo con: 49881
Entrenando con las 4 mejores + PBI + FLIA:
['metrostotales' 'precio' 'banos' 'habitaciones' 'bueno para familias']
std: 0.001360910024882652 mean: 0.24140910787760603


#### Agregamos las 2 categorias

In [112]:
cat_features = np.array(['metrostotales','precio', 'banos', 'habitaciones', 'PBI', 'bueno para familias'])


errores = cross_val_final(df_vacacional, 5, cat_features)

print(f'Dataset doble segmentacion, juntamos las 2 features:')
print(f'El dataset quedo con: {len(df_vacacional)}')
print(f'Entrenando con las 4 mejores + PBI + FLIA:')
print(cat_features)
print(f'std: {np.sqrt(np.std(errores))} mean: {np.sqrt(np.mean(errores))}')


errores = cross_val_log_final(df_vacacional, 5, cat_features)

print(f'\n\nDataset doble segmentacion, juntamos las 2 features (logaritmico):')
print(f'El dataset quedo con: {len(df_vacacional)}')
print(f'Entrenando con las 4 mejores + PBI + FLIA:')
print(cat_features)
print(f'std: {np.sqrt(np.std(errores))} mean: {np.sqrt(np.mean(errores))}')

Dataset completo, agregamos VACACIONAL:
El dataset quedo con: 49881
Entrenando con las 4 mejores + PBI + FLIA + VACACIONAL:
['metrostotales' 'precio' 'banos' 'habitaciones' 'PBI'
 'bueno para familias']
std: 2.55e+01 mean: 40.944582006935676


Dataset completo, agregamos VACACIONAL:
El dataset quedo con: 49881
Entrenando con las 4 mejores + PBI + FLIA + VACACIONAL:
['metrostotales' 'precio' 'banos' 'habitaciones' 'PBI'
 'bueno para familias']
std: 0.0008269228412811141 mean: 0.23948991614343187
